In [20]:
!apt-get -y install openjdk-8-jdk-headless
!wget http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar xf spark-2.4.0-bin-hadoop2.7.tgz
!pip install findspark kaggle pixiedust

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jdk-headless is already the newest version (8u191-b12-2ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.
--2019-03-21 01:21:25--  http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
Resolving apache.osuosl.org (apache.osuosl.org)... 64.50.233.100, 140.211.166.134, 64.50.236.52, ...
Connecting to apache.osuosl.org (apache.osuosl.org)|64.50.233.100|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 227893062 (217M) [application/x-gzip]
Saving to: ‘spark-2.4.0-bin-hadoop2.7.tgz.1’

spark-2.4.0-bin-had 100%[===================>] 217.33M  99.6MB/s    in 2.2s    

2019-03-21 01:21:27 (99.6 MB/s) - ‘spark-2.4.0-bin-hadoop2.7.tgz.1’ saved [227893062/227893062]

    100% |████████████

In [2]:
import os

# setup environment variables
os.environ["JAVA_HOME"] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ["SPARK_HOME"] = '/content/spark-2.4.0-bin-hadoop2.7'
os.environ['KAGGLE_USERNAME'] = 'ronaldsumbayak'
os.environ['KAGGLE_KEY'] = '771009dffffa0cd0883d5fb98594d756'

# verify
print(os.environ['JAVA_HOME'])
print(os.environ['SPARK_HOME'])
print(os.environ['KAGGLE_USERNAME'])
print(os.environ['KAGGLE_KEY'])

/usr/lib/jvm/java-8-openjdk-amd64
/content/spark-2.4.0-bin-hadoop2.7
ronaldsumbayak
771009dffffa0cd0883d5fb98594d756


In [3]:
# download dataset
!kaggle datasets download -d yelp-dataset/yelp-dataset
!unzip yelp-dataset.zip -d data

100% 3.59G/3.59G [01:27<00:00, 21.2MB/s]
100% 3.59G/3.59G [01:27<00:00, 43.9MB/s]
Archive:  yelp-dataset.zip
  inflating: data/yelp_academic_dataset_checkin.json  
  inflating: data/yelp_academic_dataset_user.json  
  inflating: data/yelp_academic_dataset_business.json  
  inflating: data/Dataset_Challenge_Dataset_Agreement.pdf  
  inflating: data/yelp_academic_dataset_tip.json  
  inflating: data/yelp_academic_dataset_review.json  


In [0]:
import findspark
findspark.init()

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [0]:
business = spark.read.json('data/yelp_academic_dataset_business.json')
# checkin = spark.read.json('data/yelp_academic_dataset_checkin.json')
# review = spark.read.json('data/yelp_academic_dataset_review.json')
# user = spark.read.json('data/yelp_academic_dataset_user.json')
# tip = spark.read.json('data/yelp_academic_dataset_tip.json')

In [0]:
data = business

In [9]:
data.count()

192609

In [10]:
data.show()

+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|              city|               hours|is_open|     latitude|      longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|2818 E Camino Ace...|[,,,,,,,,,,,,,,,,...|1SWheh84yJXfytovI...|   Golf, Active Life|           Phoenix|                null|      0|   33.5221425|   -112.0184807|Arizona Biltmore ...|      85016|           5|  3.0|   AZ|
|30 Eglinton Avenue W|[,, u'full_bar', ...|QXAEGFB4oINsVuTFx...|Specialty Food, R...|       Mississauga|[9:0-1:0

## Clustering

In [11]:
from pyspark.ml.feature import VectorAssembler

inputCols = ['latitude', 'longitude']
assembler = VectorAssembler(inputCols=inputCols, outputCol='features')
data = assembler.transform(data)
data.show()

+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+--------------------+
|             address|          attributes|         business_id|          categories|              city|               hours|is_open|     latitude|      longitude|                name|postal_code|review_count|stars|state|            features|
+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+--------------------+
|2818 E Camino Ace...|[,,,,,,,,,,,,,,,,...|1SWheh84yJXfytovI...|   Golf, Active Life|           Phoenix|                null|      0|   33.5221425|   -112.0184807|Arizona Biltmore ...|      85016|           5|  3.0|   AZ|[33.5221425,-112....|
|30 Eglinton Avenue W|[,, u'

In [0]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans().setK(5)
model = kmeans.fit(data)

In [23]:
predictions = model.transform(data)
predictions.show()

+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+--------------------+----------+
|             address|          attributes|         business_id|          categories|              city|               hours|is_open|     latitude|      longitude|                name|postal_code|review_count|stars|state|            features|prediction|
+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+--------------------+----------+
|2818 E Camino Ace...|[,,,,,,,,,,,,,,,,...|1SWheh84yJXfytovI...|   Golf, Active Life|           Phoenix|                null|      0|   33.5221425|   -112.0184807|Arizona Biltmore ...|      85016|           5|  3.0|   AZ|[33.5221425,-112.

In [24]:
print('Cluster centers:')
model.clusterCenters()

Cluster centers:


[array([  36.12133082, -115.17347037]),
 array([ 41.090061  , -81.00356021]),
 array([  51.04583995, -114.06720027]),
 array([  33.49261582, -111.99361954]),
 array([ 45.51148955, -73.61556547])]

In [25]:
from pyspark.ml.evaluation import ClusteringEvaluator

evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette:", silhouette)

Silhouette: 0.8039822886473685


## Visualization

In [21]:
import pixiedust

Pixiedust database opened successfully
Table VERSION_TRACKER created successfully
Table METRICS_TRACKER created successfully

Share anonymous install statistics? (opt-out instructions)

PixieDust will record metadata on its environment the next time the package is installed or updated. The data is anonymized and aggregated to help plan for future releases, and records only the following values:

{
   "data_sent": currentDate,
   "runtime": "python",
   "application_version": currentPixiedustVersion,
   "space_id": nonIdentifyingUniqueId,
   "config": {
       "repository_id": "https://github.com/ibm-watson-data-lab/pixiedust",
       "target_runtimes": ["Data Science Experience"],
       "event_id": "web",
       "event_organizer": "dev-journeys"
   }
}
You can opt out by calling pixiedust.optOut() in a new cell.


Pixiedust runtime updated. Please restart kernel
Table SPARK_PACKAGES created successfully
Table USER_PREFERENCES created successfully
Table service_connections created successfully


In [0]:
display(predictions)

![plot](imgs/plot.png)

> Note: Map view saya freeze ketika men-load datanya dan tidak menampilkan titik-titiknya sehingga saya plot menggunakan scatter plot